<a href="https://colab.research.google.com/github/ggapsang/DataStructure-C/blob/main/e3d_to_fre_230825_r1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd

# 주요 함수 선언

단위변환

In [ ]:
# mm, inch, feet 사이의 단위를 바꾼다.
def change_uom(from_uom, to_uom, value) :  #from_uom : 현재 단위,  to_uom : 바뀌는 단위, value : 값
    if from_uom == "meter" and to_uom == "feet" :
        return value*0.00328

    elif from_uom == "meter" and to_uom == "inch" :
        return value*0.03937

    elif from_uom == "feet" and to_uom == "meter" :
        return value*304.8

    elif from_uom == "feet" and to_uom == "inch" :
        return value*12

    elif from_uom == "inch" and to_uom == "meter" :
        return value*25.4

    elif from_uom == "inch" and to_uom == "feet" :
        return value*0.08333

E3D POS 좌표를 백터로 변환

In [ ]:
## (E 100000mm N200000mm U300000mm)의 형식으로 되어 있는 E3D 좌표를 백터값 (X, Y, Z)로 바꿈
def extract_coor_v(string) :
    string.strip()
    lst = string.split()
    vector_v_lst = []
    for element in range(1, 6, 2) :
        vector_v_lst.append(float(lst[element].replace("mm","")))
    vector = tuple(vector_v_lst)
    return vector

NPS에 따른 배관의 outdia 반환

In [ ]:
## NPS "값 기준으로 outdia 반환. ASTM CODE 기준. 실경의 단위는 mm이다.
def get_od(df, nps) : #df는 데이터프레임. material.csv 파일이 데이터프레임으로 변환된 상태에서 사용되는 함수
    for i in range(0, len(df)) :
        if df.loc[i, "size"] == nps :
            return df.loc[i, "outdia"]


# csv 데이터 불러오기

In [ ]:
## 데이터 파일 로딩
#df_material = pd.read_csv("material.csv") # ASTM code 기준 각종 피팅류들의 span 값 데이터
df_e3d = pd.read_csv("e3d_sample.csv") # E3D 추출값을 바탕으로 DataFrame 생성

# 데이터 전처리를 위한 자료

In [ ]:
# DataFrame에서.fre 파일 작성을 위해 필요한 칼럼 리스트
need_col = ['ARRAY ORDER',
            'NAME',
            'TYPE',
            'OWNER',
            'LBOR',
            'SPRE',
            'ITLE',
            'ITPS',
            'POS',
            'ORI',
            'ANGL']


In [ ]:
# E3D의 TYPE과 .fre의 필드 매핑 _ 1차 매핑으로 이후 형상정보들간의 관계에 따라 필드값은 달라질 수 있음
dic_type_card = {'TUBI':'TANG',
                 'VALV':'VALV',
                 'FTUB':'TANG',
                 'CAP':'CAP',
                 'ELBO':'BRAD',
                 'ATTA':'RSTN',
                 'FLAN':'FLNG',
                 'GASK':None,
                 'FILT':'ANCH',
                 'OLET':'BRAN',
                 'REDU':'ERED',
                 'INST':'VALV'}

In [ ]:
# NPS outdia 매핑. ASTM 기준
dic_nps_od = {0.5 : 21.3,
              0.75 : 26.7,
              1 : 33.4,
              1.25 : 42.2,
              1.5 : 48.8,
              2 : 60.3,
              2.5 : 73,
              3 : 88.9,
              3.5 : 101.6,
              4 : 101.6,
              5 : 141.3,
              6 : 168.3,
              8 : 219.1,
              10 : 273,
              12 : 323.8,
              14 : 355.6,
              16 : 406.4,
              18 : 457,
              20 : 508,
              22 : 559,
              24 : 610,
              26 : 660,
              28 : 711,
              30 : 762,
              32 : 813,
              34 : 864,
              36 : 914,
              40 : 1016,
              42 : 1067,
              44 : 1118,
              46 : 1168,
              48 : 1219}

In [ ]:
base_lines = [
"*==================================",
"*SECTION 1 : IDENTIFICATION & TITLE",
"*==================================",
"IDEN JB=1 CD=4 GR=-Y IU=1 OU=1",
"TITL BL=3 CV=14 GL=2 SU=1 HS=1 OF=1 BR=1 AL=3 PR=1 TI=/SKN5&6 11B15 9-371-P397-FW411 REV.02 SUBSYSTEM 9FW411/",
"",
"*==========================",
"*SECTION 2 : ANALYSIS CARDS",
"*==========================",
"",
"*MODAL EXTRACTION",
"*****************",
"FREQ FR=50 MP=1000 TR=1 LO=1 RP=8",
"",
"*STATIC CASES",
"*************",
"LCAS CA=899 TY=9 EQ=2 TI=/DESIGN WEIGHT ANALYSIS/",
"LCAS CA=10 EQ=2 TY=3 TI=/OPERATING WEIGHT ANALYSIS/",
"LCAS CA=11 EQ=2 TY=3 TI=/PEAK PRESSURE ANALYSIS/",
"LCAS CA=12 EQ=2 TY=3 TI=/DESIGN PRESSURE ANALYSIS/",
"*LCAS CA=13 EQ=2 TY=4 TI=/HYDRO TEST ANALYSIS/",
"LCAS CA=101 EQ=7 TY=0 TI=/FW CLEAN-UP RECIRCULATION (TH01)/",
"LCAS CA=102 EQ=7 TY=0 TI=/REACTOR LOW POWER NORMAL PLANT OPER/",
"LCAS CA=103 EQ=7 TY=0 TI=/NORMAL PLANT OPERATION Reactor Trip/",
"LCAS CA=201 EQ=7 TY=6 TI=/PUMP TRIP (DISP)/",
"LCAS CA=202 EQ=7 TY=6 TI=/PUMP TRIP (DISP)/",
"LCAS CA=203 EQ=7 TY=6 TI=/PUMP TRIP (DISP)/",
"LCAS CA=204 EQ=7 TY=6 TI=/PUMP TRIP (DISP)/",
"LCAS CA=205 EQ=7 TY=6 TI=/PUMP TRIP (DISP)/",
"LCAS CA=206 EQ=7 TY=6 TI=/PUMP TRIP (DISP)/",
"",
"CCAS CA=800 RF=12 ME=0 EQ=2 C1=12 TI=/EQ.8/",
"",
"CCAS CA=810 ME=8  EQ=1 C1=201 C2=202 C3=203 C4=204 C5=205 C6=206",
"CCAS CA=811 RF=11 ME=1 EQ=3 C1=11 TI=/EQ.9B/",
"CCAS CA=812 RF=11 ME=1 EQ=3 C1=11 C2=810 TI=/EQ.9B/",
"CCAS CA=813 RF=11 ME=1 EQ=3 C1=11 C2=810 TI=/EQ.9B/",
"CCAS CA=814 RF=11 ME=1 EQ=3 C1=11 C2=810 TI=/EQ.9B/",
"",
"CCAS CA=850 ME=8 EQ=7 C1=101 C2=102 C3=103 TI=/THERMAL RANGE/",
"",
"*===================================",
"*SECTION 3: GEOMETRY & LOADING CARDS",
"*===================================",
"*",
"*MATERIALS",
"**********",
"* MATERIAL: A106 Grade B (C-Si steel)",
"***************************************",
"MATH CD=107 EX=2 TY=9 SG=0.3 TA=70 TX=700 KL=1",
"MATD TE=70  EH=29.5 EX=6.40 SH=17.1 SY=35.0",
"MATD TE=100 EH=29.5 EX=6.40 SH=17.1 SY=35.0",
"MATD TE=200 EH=28.8 EX=6.70 SH=17.1 SY=32.1",
"MATD TE=300 EH=28.3 EX=6.90 SH=17.1 SY=31.0",
"MATD TE=400 EH=27.7 EX=7.10 SH=17.1 SY=29.9",
"MATD TE=500 EH=27.3 EX=7.30 SH=17.1 SY=28.5",
"MATD TE=600 EH=26.7 EX=7.40 SH=17.1 SY=26.8",
"MATD TE=700 EH=25.5 EX=7.60 SH=15.6 SY=25.1",
"",
"",
"* MATERIAL: A358-304 CL2 (18Cr - 8Ni)",
"************************************",
"MATH CD=108 EX=2 TY=9 SG=0.3 TA=70 TX=800 KL=1",
"MATD TE=70  EH=28.3 EX=8.5 SH=16.2  SY=30.0",
"MATD TE=200 EH=27.6 EX=8.9 SH=16.2  SY=25.0",
"MATD TE=300 EH=27.0 EX=9.2 SH=15.3  SY=22.4",
"MATD TE=400 EH=26.5 EX=9.5 SH=14.8  SY=20.7",
"MATD TE=500 EH=25.8 EX=9.7 SH=14.1  SY=19.4",
"MATD TE=600 EH=25.3 EX=9.8 SH=14.0  SY=18.4",
"MATD TE=650 EH=25.1 EX=9.9 SH=13.1  SY=18.0",
"MATD TE=700 EH=24.8 EX=10.0 SH=12.8 SY=17.2",
"MATD TE=800 EH=24.1 EX=10.1 SH=12.3 SY=16.9",
"",
"",
"*CROSS SECTIONS",
"***************",
"CROS CD=1 OD=14 WT=0.938 MA=195.83 KL=1 AL=/14\" PIPE SCH.100/",
"CROS CD=2 OD=14 WT=3.75 KL=1 AL=/10\" GLOBE VALVE(V0107)/",
"CROS CD=3 OD=8.625 WT=0.594 MA=80.834 KL=1 AL=/8\" PIPE SCH.100 INSUL2.5/",
"CROS CD=4 OD=8.625 WT=2.79 KL=1 AL=/8\" GATE VALVE (V2200, V2201)/",
"CROS CD=5 OD=24 WT=1.812 KL=1 AL=/24\" HEADER PIPE/",
"CROS CD=6 OD=14 WT=0.75 MA=174.38 KL=1 AL=/14\" PIPE SCH.80/",
"CROS CD=7 OD=10.75 WT=0.719 MA=118.93 KL=1 AL=/10\" PIPE SCH.100/",
"",
"*INITIAL CONDITIONS",
"*******************",
"MATL CD=108",
"",
"DESN TE=400 PR=500",
"PRES CA=11  PR=550  AL=/PEAK PRESSURE/,",
"PRES CA=12  PR=750  AL=/DESIGN PRESSURE/,",
"*PRES CA=13  PR=750  AL=/HYDRO TEST PRESSURE/",
"OPER CA=101 TE=228  AL=/FW CLEAN-UP RECIRCULATION (TH01)/ ",
"OPER CA=102 TE=91.7 AL=/NORMAL PLANT OPER/",
"OPER CA=103 TE=91.7 AL=/Reactor Trip/",
""
]

# 1차 데이터 전처리 : E3D 데이터 중 불필요한 칼럼 삭제

In [ ]:
# E3D Data의 해더 리스트
sample_hd = list(df_e3d)

# .fre 작성에 필요 없는 칼럼들의 리스트, need_col에 들어있는 값 이외의 해더값 삭제
drop_cols = []
for col in sample_hd :
    if col not in need_col :
        drop_cols.append(col)
df_e3d = df_e3d.drop(drop_cols, axis=1)

# 2차 데이터 전처리 : E3D 데이터에 .fre 파일 작성을 위해 필요한 칼럼 및 값들 추가

POINT IDENTITY 값 생성

In [ ]:
# "PT=" FIELD(노드값)에 들어갈 값 "PT=20"부터 시작하여 10씩 증가하는 방식으로 node번호를 매긴다.
# (Peps의 node값(point identitiy)는 응력 해석의 기준점으로 반드시 형상의 개수와 일치하는 것이 아니며 사용자가 필요에 따라 추가할 수 있어야 하므로 10의 배수로 끊어서 작성한다)
# E3D 데이터에서 ARRYA_ORDER가 정수가 아닌 실수인 경우(BRANCH의 DR/VT 등) 바로 앞의 번호 뒤에 A,B,C,D..Z순으로 채번한다

pt_lst = []
n = 1
# 데이터의 'ARRAY ORDER' 칼럼을 순회하며 point identity 값을 생성하여 리스트에 넣는다
for i in range(0, len(df_e3d)) :
    # pt값이 정수인 경우
    if  df_e3d.loc[i,'ARRAY ORDER'].astype(int) == df_e3d.loc[i,'ARRAY ORDER']:
        pt_lst.append(n*10)
        n +=1
    # pt값이 실수인 경우
    else :
        tup = np.modf(df_e3d.loc[i,'ARRAY ORDER'])
        pt = str(n*10) + chr(int(np.round(tup[0]*10, 1) + 64))
        pt_lst.append(pt)

df_e3d['PT='] = pt_lst #e3d 데이터 프레임에 "PT=" 열과 값 추가

CARD 값 생성

In [ ]:
# "CARD"(.fre 스크립트의 매 줄 첫머리에 들어갈 명령어)

card_lst = []
# E3D 데이터의 'TYPE' 칼럼을 순회하며 "CARD="값을 생성하여 리스트에 넣는다
for i in range(0, len(df_e3d)) :
    card_lst.append(dic_type_card[df_e3d.loc[i, 'TYPE']])
df_e3d['CARD'] = card_lst

위치벡터 값 생성

In [ ]:
coor_lst = []
for i in range(0, len(df_e3d)) :
    try :
        coor_lst.append(extract_coor_v(df_e3d.loc[i, 'POS']))
    # 'POS' 칼럼의 값들을 기본으로 참조하지만, 해당 값이 결측치일 경우 'IPTS'의 값을 2차로 참조한다
    except:
        coor_lst.append(extract_coor_v(df_e3d.loc[i, 'ITPS']))
df_e3d['COOR'] = coor_lst

joint type 값 생성

In [ ]:
# 'EW=' component에 용접타입이 명시되어 있지 않은 경우 2"미만 sw(3), 2"이상은 bw(2)로 잡는다)
ew_lst = []
for i in range(0, len(df_e3d)) :
    if float(df_e3d.loc[i, "LBOR"].replace("mm", "")) < 50 :
        ew_lst.append(3)
    else :
        ew_lst.append(2)

df_e3d["EW="] = ew_lst

기준점 벡터부터 시작하여 변위차 계산 및 좌표계 단위 환산

In [ ]:
# component array order에 따른 △v
dv_lst = []
for ele in range(0, len(df_e3d)) :
    if ele == 0 :
        dv = (0, 0, 0)
        dv_lst.append(dv)
    else :
        dv = tuple(np.around(np.array(df_e3d.loc[ele, 'COOR']) - np.array(df_e3d.loc[ele-1, 'COOR']), decimals=10))
        dv_lst.append(dv)
df_e3d["dv"] = dv_lst

# dx, dy, dz 좌표 단위 변환(필요시)
dx_lst_cnv = [ change_uom(from_uom='meter', to_uom='feet', value = df_e3d.loc[i, 'dv'][0]) for i in range(0, len(df_e3d)) ]
dy_lst_cnv = [ change_uom(from_uom='meter', to_uom='feet', value = df_e3d.loc[i, 'dv'][1]) for i in range(0, len(df_e3d)) ]
dz_lst_cnv = [ change_uom(from_uom='meter', to_uom='feet', value = df_e3d.loc[i, 'dv'][2]) for i in range(0, len(df_e3d)) ]
df_e3d["DX="] = dx_lst_cnv
df_e3d["DY="] = dy_lst_cnv
df_e3d["DZ="] = dz_lst_cnv

RA 값 생성

In [ ]:
####### 먼저 E3D의 LBOR 값을 "(인치)기준 NPS로 바꾼다
nps_lst = []
for i in range(0, len(df_e3d)) :
    if int(df_e3d.loc[i, "LBOR"].replace("mm", "")) == 15 :
        nps_lst.append(0.5)
    elif int(df_e3d.loc[i, "LBOR"].replace("mm", "")) == 20 :
        nps_lst.append(0.75)
    elif int(df_e3d.loc[i, "LBOR"].replace("mm", "")) == 32 :
        nps_lst.append(1.25)
    elif int(df_e3d.loc[i, "LBOR"].replace("mm", "")) == 40 :
        nps_lst.append(1.5)
    elif int(df_e3d.loc[i, "LBOR"].replace("mm", "")) == 65 :
        nps_lst.append(2.5)
    elif int(df_e3d.loc[i, "LBOR"].replace("mm", "")) == 80 :
        nps_lst.append(3)
    else :
        nps_lst.append(int(df_e3d.loc[i, "LBOR"].replace("mm", ""))/25)

df_e3d["NPS"] = nps_lst


In [ ]:
# E3D에 추가한 CARD 열의 값 중 "BRAD"에 RA값을 집어넣는다. (다른 CARD들의 RA 값은 0또는 결측치로 둔다)
    # 참고 :
        # 엘보의 RAIDUS를 계산하기 위해서는 3가지 값을 알아야 한다
            # 1. 소위 '엘보값'이라 불리는 엘보 끝단부터 센터까지의 거리. 아래 코드에서는 'span'이라는 변수에 할당함. LONG ELBOW와 SHORT ELBOW의 값이 다르고, sw와 bw의 값이 다르다
            # 2. NPS에 따른 outdia(엘보의 외경) 값. 위에서 먼저 구해 놓은 값으로 계산한다
            # 3. 엘보의 각도 : 엘보값과 RADIUS값을 계산할 때 모두 사용한다

ra_lst = []
for i in range(0, len(df_e3d)) :
    if df_e3d.loc[i, 'CARD'] != "BRAD" :
        ra_lst.append(0)

    else :
        # E3D 데이터 'ANGL' 열의 값에서 찾음
        degree_str = df_e3d.loc[i, "ANGL"]
        degree = float(degree_str.replace("degree", "") if degree_str != "-" else 0)

        # 바로 앞에서 추가한 NPS 값
        nps = df_e3d.loc[i, "NPS"]
        #od = get_od(df_material, nps)
        od = dic_nps_od[nps]

        # LONG ELBOW로 가정함
        span = 1.5 * 25.4 * np.tan(degree / 2) #LONG ELBOW, 2인치 이상의 BW일때 엘보값 계산 공식 (1.5 * 25.4 * tan(각도/2), short엘보일 경우 1.5를 곱하지 않음)
        ra = span * (1 / np.sin(degree / 2)) * 0.5 * od # RADIUS 계산 공식 (엘보값 * 1/sin(각도/2) * 엘보 반지름)
        ra_cnv = np.round(change_uom('meter', 'feet', ra), 3) #피트로 단위변환
        ra_lst.append(ra_cnv)

df_e3d["RA="] = ra_lst

전처리를 끝낸 E3D 데이터

In [ ]:
#df_e3d

추가된 코드 BRAN TE 값

In [ ]:
#te_lst = []

# .fre 파일 작성

Geometry 이전 부분의 작성

In [ ]:
###################### .fre 파일 작성 ##############################################
#basic_form = open("base_format.txt", 'rt', encoding='UTF8')
#basic_contents = tuple([line for line in basic_form.readlines()])
#basic_form.close()

#######GEOMETRY의 첫 행
final_output = open("sample_0824.fre", 'w')

for line in base_lines :
    final_output.write(line)

line = "\n" "COOR PT=" + str(df_e3d.loc[0, "PT="]) + " " \
    + "AX=" + str(change_uom("meter", "feet", df_e3d.loc[0, "COOR"][0])) + " " \
    + "AY=" + str(change_uom("meter", "feet", df_e3d.loc[0, "COOR"][1])) + " " \
    + "AZ=" + str(change_uom("meter", "feet", df_e3d.loc[0, "COOR"][2])) + "\n"
final_output.write(line)

line ="ANCH " + "PT=" + str(df_e3d.loc[0, "PT="]) + " TA=2" + " LV=1" + '\n'

final_output.write(line)
final_output.close()

main line 먼저 작성

In [ ]:
#GEOMETRY의 main line
final_output = open("sample_0824.fre", 'a')
branch_point_lst = []
for i in range(0, len(df_e3d)):
    main_or_bran = True if df_e3d.loc[i,'ARRAY ORDER'].astype(int) == df_e3d.loc[i,'ARRAY ORDER'] else False
    word_lst = []
    card = df_e3d.loc[i, "CARD"]
    point_id = df_e3d.loc[i, "PT="]
    dx = df_e3d.loc[i, "DX="]
    dy = df_e3d.loc[i, "DY="]
    dz = df_e3d.loc[i, "DZ="]

    if main_or_bran == True :

        if card == "BRAD" :
            word_lst.append("BRAD")

            word_lst.append("PT=" + str(point_id))

            word_lst.append("RA=" + str(df_e3d.loc[i, "RA="]))

            word_lst.append("EW=" + str(df_e3d.loc[i, "EW="]))

            final_output.write(" ".join(word_lst) + '\n')

        # branch table 보강, junc 이후 분기점에 대한 처리방법 보강
        elif card == "BRAN" :
            word_lst.append("BRAN")

            word_lst.append("PT=" + str(point_id))

            if dx != 0 :
                word_lst.append("DX=" + str(dx))

            if dy != 0 :
                word_lst.append("DY=" + str(dy))

            if dz != 0 :
                word_lst.append("D=Z" + str(dz))

            # case별로 te 분석하여 "TE=" table 작성

            word_lst.append("TE=6")

            word_lst.append("EW=" + str(df_e3d.loc[i, "EW="]))

            # branch point list에 추가 이후 가지관 위치 서술시 필요함
            branch_point_lst.append(point_id)

            final_output.write(" ".join(word_lst) + '\n')

        elif card != "RSTN" and card !=None :
            word_lst.append(card)

            word_lst.append("PT=" + str(point_id))

            word_lst.append("EW=" + str(df_e3d.loc[i, "EW="]))

            final_output.write(" ".join(word_lst) + '\n')

        elif card == "RSTN" :
            word_lst.append(card)

            word_lst.append("PT=" + str(point_id))

            word_lst.append("EW=" + str(df_e3d.loc[i, "EW="]))

            final_output.write(" ".join(word_lst) + '\n')

        else :
            continue

    else :
        continue

# 종료 구문 작성 후 파일 닫
final_output.write("ENDP")
final_output.close()

In [ ]:
branch_point_lst

[70, 150, 310, 320]

In [ ]:
final_output = open("sample_0824.fre", 'a')

condition = (df_e3d.loc[i,'ARRAY ORDER'].astype(int) != df_e3d.loc[i,'ARRAY ORDER'])



final_output.close()

In [ ]:

######## GEOMETRY의 BRNACH, DR, VT

### 추가로 csv 파일(참고용)
df_e3d.to_csv("final_df.csv", index=False)